# T1 

In [1]:
import datetime

# Define the objective function to minimize (MSE)
def objective(params, X_test, y_test):
    predicted = np.dot(X_test, params)
    mse = np.mean((predicted - y_test) ** 2)
    return mse

og_predictions_today = {} 
og_predictions_tomorrow = {} 

current_datetime = datetime.datetime.now()
current_date_str = current_datetime.strftime('%Y-%m-%d %H:%M:%S %Z')


def original_optimized_linear_regress():
    print('Beginning Optimized Standard Linear Regression')
    print('')
    
    # for pair, features_df in features_dict.items():
    for pair, features_df in head_features.items(): 
        # Prepare the data
        X = features_df.drop(columns = ['close']).values[:-2]  # Exclude last two values for today's and tomorrow's prediction
        y = features_df['close'].values[:-2]  # Similarly, exclude the last two values 

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            
        # Optimization with basinhopping
        initial_params = np.ones(X_test.shape[1])
        result = opt.basinhopping(objective, initial_params, niter=100, stepsize=0.5, minimizer_kwargs={'args': (X_test, y_test)})
        optimized_params = result.x
        # Using the optimized parameters to make predictions
        og_prediction_today = np.dot(features_df.drop(columns = ['close']).iloc[-2, :].values, optimized_params)
        og_prediction_tomorrow = np.dot(features_df.drop(columns = ['close']).iloc[-1, :].values, optimized_params)

        og_predictions_today[pair] = prediction_today
        og_predictions_tomorrow[pair] = prediction_tomorrow
 
    print(f"Time is {current_date_str}.")
    print('')
    
    for pair in predictions_today.keys():
        first_pair, second_pair = pair  # split the pair into individual currencies
        
        current_price = round(compute_spread(pair).iloc[-1], 5)
        print(f"For {pair}:")
        print(f"Today's prediction: {og_predictions_today[pair]:.5f}. Current price: {current_price}")
        
        if og_predictions_today[pair] > current_price:  # predicted spread is widening
            print(f"Signal: Sell {first_pair}, Buy {second_pair}")
            orders["sell"].append(first_pair)
            orders["buy"].append(second_pair)
        elif og_predictions_today[pair] < current_price:  # predicted spread is contracting
            print(f"Signal: Buy {first_pair}, Sell {second_pair}")
            orders["buy"].append(first_pair)
            orders["sell"].append(second_pair)
        print("-----")

original_optimized_linear_regress()

Beginning Optimized Standard Linear Regression



NameError: name 'head_features' is not defined

In [ ]:
class MarkovChain:
    
    # Adjust Accordingly 
    def __init__(self, price_data):
        self.data = pd.DataFrame(price_data)
        self.states = ['s_bull', 'w_bull', 'doji', 'w_bear', 's_bear']
    
    def compute_returns(self):
        self.data['returns'] = self.data['close'].pct_change()
        
    # Temporary states, to amend to more sophisticated/specific states
    def classify_state(self, ret):
        if ret > 0.01:
            return 's_bull'
        elif ret > 0.003 and ret < 0.01:
            return 'w_bull'
        elif ret < 0.002 and ret > -0.002:
            return 'doji'
        elif ret < -0.003 and ret > -0.01:
            return 'w_bear'
        else:
            return 's_bear'
        
    def classify_states(self):
        self.data['state'] = self.data['returns'].apply(self.classify_state)
        
        
    # ChatGPT code testing/creating a matrix to test for ergodicity 
    def initialize_matrix(self):
        self.matrix = np.zeros((len(self.states), len(self.states)))

    def construct_transition_matrix(self):
        self.initialize_matrix()

        for i in range(1, len(self.data['state'])):
            curr_state = self.data['state'][i]
            prev_state = self.data['state'][i - 1]
            curr_index = self.states.index(curr_state)
            prev_index = self.states.index(prev_state)

            self.matrix[prev_index][curr_index] += 1

        # Normalize the matrix to get transition probabilities
        self.matrix = self.matrix / self.matrix.sum(axis=1)[:, None]

    def is_ergodic(self):
        if (self.matrix > 0).all():
            return True
        return False

In [ ]:
from sklearn.neighbors import NearestNeighbors

nearest_neighbor_models = {}  # A dictionary to hold the trained NearestNeighbors models for each pair.

def classify_into_state(features, pair):
    """Classify a given prediction into a state using the NN and DBSCAN."""
    activation = states_dict[pair]["activation_model"].predict(features)
    
    if pair not in nearest_neighbor_models:
        # If NearestNeighbors model hasn't been trained for this pair, train it
        train_activations = states_dict[pair]["activation_model"].predict(features_dict[pair])
        nearest_neighbor_models[pair] = NearestNeighbors(n_neighbors=1).fit(train_activations)

    distance, index = nearest_neighbor_models[pair].kneighbors(activation)
    
    states = states_dict[pair]["states"]
    state = states[index[0][0]]  # Get the state of the nearest neighbor
    
    return state

def update_transition_matrix(current_state, previous_state, transition_matrix):
    """Update the given transition matrix based on the transition from previous_state to current_state."""
    
    # Check if previous_state exists in transition_matrix
    if previous_state not in transition_matrix:
        transition_matrix[previous_state] = {}

    # Now update or initialize current_state for the given previous_state
    if current_state not in transition_matrix[previous_state]:
        transition_matrix[previous_state][current_state] = 0
        
    transition_matrix[previous_state][current_state] += 1

### Linear Regress

In [ ]:

# Define the objective function to minimize (MSE)
def objective(params):
    predicted = np.dot(X_test, params)
    mse = np.mean((predicted - y_test) ** 2)
    return mse

predictions = {} 

for pair, features_df in features_dict.items():
    
    # Prepare the data
    X = features_df.values  # Feature values
    y = coint_dict[pair].values[59:]  # All values as target

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a linear regression model as an example
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    # print(f"Mean Squared Error for {pair}: {mse}")
    
     # Initialize the parameters for optimization (e.g., as all ones)
    initial_params = np.ones(X_test.shape[1])

    # Use simulated annealing to optimize the parameters
    result = opt.basinhopping(objective, initial_params, niter=100, stepsize=0.5)

    # Get the optimized parameters
    optimized_params = result.x

    # Re-predict using the optimized parameters
    y_pred_optimized = np.dot(X_test, optimized_params)

    # Save the prediction
    predictions[pair] = y_pred_optimized[0]

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred_optimized)
    r_squared_optimized = r2_score(y_test, y_pred_optimized)
    print(f"Mean Squared Error for {pair}: {mse}")
    # print(f'R-squared (R^2) for {pair}: {r_squared_optimized}')

print(predictions)

In [ ]:
import datetime

# Define the objective function to minimize (MSE)
def objective(params):
    predicted = np.dot(X_test, params)
    mse = np.mean((predicted - y_test) ** 2)
    return mse

predictions_today = {} 
predictions_tomorrow = {} 
change_in_predictions = {}

current_datetime = datetime.datetime.now()
current_date_str = current_datetime.strftime('%Y-%m-%d %H:%M:%S %Z')

def standard_linear_regress():
    
    print('Beginning Normal Standard Linear Regression')
    print('')
    for pair, features_df in features_dict.items():
        # Prepare the data
        X = features_df.values[:-2]  # Exclude last two values for today's and tomorrow's prediction
        y = coint_dict[pair].values[59:-2]  # Similarly, exclude the last two values 

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train a linear regression model
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Predict for today and tomorrow using the most recent features
        prediction_today = model.predict([features_df.iloc[-2, :].values])[0]
        prediction_tomorrow = model.predict([features_df.iloc[-1, :].values])[0]

        predictions_today[pair] = prediction_today
        predictions_tomorrow[pair] = prediction_tomorrow
        change_in_predictions[pair] = prediction_today - model.predict([features_df.iloc[-3, :].values])[0]

    print(f"Time is {current_date_str}.")
    for pair in predictions_today.keys():
        current_price = round(compute_spread(pair).iloc[-1], 5)
        print(f"For {pair}:")
        print(f"Today's prediction: {predictions_today[pair]:.5f}. Current price: {current_price}")
        print(f"Tomorrow's prediction: {predictions_tomorrow[pair]:.5f}")
        print(f"Change in prediction compared to yesterday: {change_in_predictions[pair]:.5f}")
        print("-----")

standard_linear_regress()

### ARIMA

In [ ]:
#TESTING
for pair, data in coint_dict.items():
    print(f"Running through {pair}")
    split = int(len(data)*0.80)
    train_set, test_set = data[:split], data[split:]
    
    model = ARIMA(train_set, order=(2, 1, 2))
    model_fit_0 = model.fit()
    
    # Convert pandas series to list for rolling window forecast
    past = train_set.tolist()

    # Empty list for storing predictions
    predictions = []

    # Keeping only the first 50 data in the test dataset.
    # You can run on the whole dataset, but it will take time to run.
    test_set = test_set[:50]

    # Perform rolling window forecast
    for i in range(len(test_set)):
        # Define ARIMA model
        model = ARIMA(past, order=(2, 1, 2))
        # Fit the model
        model_fit = model.fit(start_params=model_fit_0.params)
        # Make forecast
        forecast_results = model_fit.forecast()
        pred = forecast_results[0]
        # Append prediction
        predictions.append(pred)
        # Add test value to train set
        past.append(test_set[i])

NameError: name 'coint_dict' is not defined

In [ ]:
# AR AND MA
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
plt.style.use('seaborn-darkgrid')

for pair, data in coint_dict.items():
    print(f"Running through {pair}")
    split = int(len(data)*0.80)
    train_set, test_set = data[:split], data[split:]

    # Empty list to store aic/bic score
    aic_p = []
    bic_p = []

    # p values
    p = range(1, 6)  # [1,2,3,4,5]

    # AIC/BIC score for different values of p
    for i in p:
        # Define the AR model
        model = ARIMA(train_set, order=(i, 1, 0))  
        # Fit the model
        model_fit = model.fit()
        # Get AIC score
        aic_temp = model_fit.aic  
        # Get BIC score
        bic_temp = model_fit.bic
        # Append AIC score
        aic_p.append(aic_temp) 
        # Append BIC score
        bic_p.append(bic_temp) 

    # Plot of AIC/BIC score for AR term
    plt.figure(figsize=(15, 7))
    plt.plot(range(1, 6), aic_p, color='red')
    plt.plot(range(1, 6), bic_p)
    plt.title('Tuning AR term')
    plt.xlabel('p (AR term)')
    plt.ylabel('AIC/BIC score')
    plt.legend(['AIC score', 'BIC score'])
    plt.show() 
    
for pair, data in coint_dict.items():
    print(f"Running through {pair}")
    split = int(len(data)*0.80)
    train_set, test_set = data[:split], data[split:]

    # Empty list to store AIC/BIC score
    aic_q = []
    bic_q = []

    # q values
    q = range(1, 6)

    # AIC/BIC score for different values of q
    for i in q:
        model = ARIMA(train_set, order=(0, 1, i))
        model_fit = model.fit()
        aic_temp = model_fit.aic
        bic_temp = model_fit.bic
        aic_q.append(aic_temp)
        bic_q.append(bic_temp)

    # Plot of AIC/BIC score for MA term
    plt.figure(figsize=(15, 7))
    plt.plot(range(1, 6), aic_q, color='red')
    plt.plot(range(1, 6), bic_q)
    plt.title('Tuning MA term')
    plt.xlabel('q (MA term)')
    plt.ylabel('AIC/BIC score')
    plt.legend(['AIC score', 'BIC score'])
    plt.show()

In [ ]:
# Calculating Features #

def nn_calc_features(pair1, pair2, timeframe, x):
    # You'll need to fetch pair1's data in a similar manner as pair2
    # Assuming pair1 is fetched similarly as below
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair2 = pd.DataFrame(mt5.copy_rates_from_pos(pair2, timeframe, 0, x))
    df = pair1[['open', 'high', 'low', 'close']] - pair2[['open', 'high', 'low', 'close']]
    
    fiften_day_avg = df['close'].rolling(window = 15).mean().round(5)
    sixty_day_avg =  df['close'].rolling(window= 60).mean().round(5)
    
    df['fiften_day_avg'] = fiften_day_avg
    df['sixty_day_avg'] = sixty_day_avg
    
    return df.dropna()

nn_features = {}
for pair in coint_dict.keys():
    pair1 = pair[0]
    pair2 = pair[1]
    diff = nn_calc_features(pair1, pair2, mt5.TIMEFRAME_D1, 1000)
    nn_features[pair] = diff

In [ ]:
class MarkovChain:
    def __init__(self, states, states_dict):
        self.states = states
        self.transition_matrix = {}
        self.classifier = StateClassifier(states_dict)
        self.previous_state = None
    
    def update_transition_matrix(self, current_state, pair):
        # Check and initialize the outer dictionary for the pair if needed
        if pair not in self.transition_matrix:
            self.transition_matrix[pair] = {}

        # Check and initialize the second level dictionary for previous_state if needed
        if self.previous_state not in self.transition_matrix[pair]:
            self.transition_matrix[pair][self.previous_state] = {}

        # Check and initialize the innermost dictionary for current_state if needed
        if current_state not in self.transition_matrix[pair][self.previous_state]:
            self.transition_matrix[pair][self.previous_state][current_state] = 0

        # Now you can safely update the count
        self.transition_matrix[pair][self.previous_state][current_state] += 1

    def classify_samples(self, samples, pair):
        return [self.classifier.classify_sample(sample.reshape(1, -1), pair) for sample in samples]

    def get_transition_matrix(self):
        return self.transition_matrix
    
    def create_transition_matrix(self, transitions_dict):

        result = {}

        for pair, transitions in transitions_dict.items():
            transition_counts = {}
            for from_state, to_states in transitions.items():
                for to_state, count in to_states.items():
                    if from_state not in transition_counts:
                        transition_counts[from_state] = {}
                    if to_state not in transition_counts[from_state]:
                        transition_counts[from_state][to_state] = 0
                    transition_counts[from_state][to_state] += count

            probability_matrix = {}
            for from_state, to_states in transition_counts.items():
                total_transitions = sum(to_states.values())
                probability_matrix[from_state] = {to_state: count / total_transitions for to_state, count in to_states.items()}

            result[pair] = probability_matrix

        return result

In [ ]:
# Initialize a list to store the statistics
statistics_list = []

# Calculate statistics for each group
for sub_state, pct_changes in grouped:
    mean_values = pct_changes.mean()
    mode_values = pct_changes.mode()[0] if not pct_changes.mode().empty else np.nan
    std_values = pct_changes.std()
    var_values = pct_changes.var()
    kurt_values = kurtosis(pct_changes, fisher=True)
    
    fifteen_MA = pct_changes.rolling(window=15).mean()
    forty_five_MA = pct_changes.rolling(window=45).mean()
    ratio = fifteen_MA / forty_five_MA
    
    q1 = ratio.quantile(0.25)
    q3 = ratio.quantile(0.75)
    iqr = q3 - q1
    
    # Append statistics to the list
    statistics_list.append({
        'sub_state': sub_state,
        'Mean': mean_values,
        'Mode': mode_values,
        'STD': std_values,
        'VAR': var_values,
        'Kurtosis': kurt_values,
        'Q1': q1,
        'Q3': q3,
        'IQR': iqr
    })

# Convert the list of dictionaries to a DataFrame
statistics_df = pd.DataFrame(statistics_list)

# Now, split the data
X = df[['pct_change', 'RSI']].dropna()
y = df['Label'].dropna()
sub_states = df['sub_state']

X_train, X_test, y_train, y_test, sub_states_train, sub_states_test = train_test_split(
    X, y, sub_states, test_size=0.2, random_state=42
)
# Step 1: Fit a Random Forest model to determine feature importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importances
importances = rf.feature_importances_

# Step 2: Select important features based on feature importances
# (Here, we're arbitrarily choosing to keep the top 10 features)
important_feature_indices = np.argsort(importances)[::-1][:10]


# Extract the important features from the original data
important_feature_names = X_train.columns[important_feature_indices]
X_train_important = X_train[important_feature_names]
X_test_important = X_test[important_feature_names]

# Step 3: Train an SVM model using only important features
svm = SVC(kernel='linear', C=1, probability=True)  # Note the `probability=True`
svm.fit(X_train_important, y_train)

# Evaluate the model (Optional)
score = svm.score(X_test_important, y_test)
print(f"SVM model accuracy: {score * 100:.2f}%")

# Step 4: Get confidence scores (continuous values between 0 and 1)
confidence_scores = svm.predict_proba(X_test_important)[:, 1]

# Normalize the confidence_scores to be between 0 and 1
scaler = MinMaxScaler((0, 1))
confidence_scores = scaler.fit_transform(confidence_scores.reshape(-1, 1))
confidence_scores = confidence_scores.flatten()
confidence_scores_df = pd.DataFrame({'Confidence_Score': confidence_scores, 'sub_state': sub_states_test.reset_index(drop=True)})
aggregated_scores = confidence_scores_df.groupby('sub_state')['Confidence_Score'].mean()

# Convert to DataFrame for better visualization
confidence_df = pd.DataFrame({'Confidence_Score': confidence_scores.flatten()})
print(confidence_df.head())

statistics_df['SVM_Score'] = statistics_df['sub_state'].map(aggregated_scores)

# Calculate scores
statistics_df['Score'] = statistics_df.apply(calculate_score, axis=1)

statistics_df

In [ ]:
        # Transition Matrix Creation 
        
#         prediction_features = features_df.iloc[:-2]
#         current_state = classify_into_state(prediction_features, pair)
#         print(current_state)
        
#         if previous_state is not None:
#             print('Updating Transition Matrix')
#             update_transition_matrix(current_state, previous_state, transition_matrix)
#         else:
#             print('No Updates')

#         previous_state = current_state
#       # Normalize the transition matrix 
#         counter = 0
#         for state_from, transitions in transition_matrix.items():
#             counter += 1
#             print(counter)
#             total_transitions = sum(transitions.values())
#             if total_transitions > 0:
#                 transition_matrix[state_from] = {state_to: count / total_transitions for state_to, count in transitions.items()}